# Project By Raunaq Singh Jabbal

Project made considering time constraints, is easy to read and understand, with less LOC's and model has fast inference.

# 0. Imports

In [ ]:
import bs4 as BeautifulSoup
import selenium
import time
from sentence_transformers import SentenceTransformer, util
import torch
import numpy as np
import pandas as pd
import unidecode
from word2number import w2n
import contractions
import regex as re


# 1. Web Scraping and PreProcessing

In [ ]:
# Preprocessing Function
def preprocess(text):
    text = unidecode.unidecode(text)    # Remove accents
    text = contractions.fix(text)       # Expand contractions
    text = text.lower()                 # Convert to lowercase
    # re.sub(r'\d+', '', text)          # Remove numbers
    text = re.sub(r'[^\w\s]', '', text) # Remove punctuation
    text = text.strip()                 # Remove extra whitespaces
    text = " ".join(text.split())       
    return text

In [1]:
N=10                                                                            # Number of items to be displayed

model = SentenceTransformer('all-mpnet-base-v2')                                # Loading the Sentence Transformer Model

dataset = pd.read_csv('ajio.csv')                                               # Loading the Datastore from the CSV file

embeddings = model.encode(dataset["Title"], convert_to_tensor=True)             # Getting the embeddings of the Search Query


pd.DataFrame(embeddings.numpy()).to_csv('embeddings.csv', index=False)          # Saving the embeddings in a CSV file